In [1]:
import pandas as pd
import numpy as np

In [2]:
totalrides = pd.read_csv('totalrides.csv')
tripduration = pd.read_csv('tripdurationcount.csv')

In [3]:
totalrides

,total_rides,month,year
0,577702,6,2013
1,843417,7,2013
2,1001958,8,2013
3,1034359,9,2013
4,1037712,10,2013
...,...,...,...
87,2488225,9,2020
88,2248869,10,2020
89,1736704,11,2020
90,1088929,12,2020


In [33]:
# this was number of rides that lasted for a specific duration in seconds
# for example there were 2,584 rides that lasted 60 seconds.. 17,009 rides that lasted 61 seconds

tripduration


,count,trip_duration
0,2584,60
1,17009,61
2,17000,62
3,17262,63
4,17563,64
...,...,...
92820,1,13586276
92821,1,13931824
92822,1,15020934
92823,1,15962256


In [54]:
# counting by minutes
# for example there were 616,061 rides that lasted approximately a minute

tripduration['durationinmin'] = (tripduration['trip_duration']/60).round(0)

tripdurationbymin = tripduration.groupby('durationinmin')['count'].sum()
tripdurationbymin = pd.DataFrame(tripdurationbymin).reset_index()

tripdurationbymin

,durationinmin,count
0,1.0,616061
1,2.0,2733559
2,3.0,4743085
3,4.0,6530210
4,5.0,7141123
...,...,...
13034,226438.0,1
13035,232197.0,1
13036,250349.0,1
13037,266038.0,1


In [189]:
import plotly.express as px

# taking a look at the distribution of rides that were at most an hour
final = tripdurationbymin[tripdurationbymin['durationinmin'] <= 60]
import pickle
pickle.dump(final, open('countofridesbyminutes.pkl', 'wb'))

fig = px.bar(final, 
             x="durationinmin", 
             y="count", 
             title="Trip Duration Count",
             labels = dict(count="Total Number of Rides", durationinmin="Minutes"))
fig.update_traces(marker_color='blue')
for data in fig.data:
   data["width"] = 1.0 
fig.layout.plot_bgcolor = 'white'
fig.update_yaxes(nticks = 16, ticks = 'outside', showgrid = False)
fig.update_xaxes(nticks = 14, ticks = 'outside')

fig.show()

In [100]:
totalrides["monthyear"] = totalrides["month"].astype(str) + "/" + totalrides["year"].astype(str)
totalrides["monthyear"]

0      6/2013
1      7/2013
2      8/2013
3      9/2013
4     10/2013
       ...   
87     9/2020
88    10/2020
89    11/2020
90    12/2020
91     1/2021
Name: monthyear, Length: 92, dtype: object

In [111]:
fig = px.line(totalrides, 
              x='monthyear', 
              y='total_rides', 
              title="Monthly Total Rides",
              labels = dict(total_rides="Total Number of Rides", monthyear="Month/Year"),
              markers=True)
fig.layout.plot_bgcolor = 'white'
fig.update_traces(marker={'size': 12,
                          'color': 'blue'})
fig.update_yaxes(nticks=15, ticks = 'outside', showgrid = False)
fig.update_xaxes(nticks = 15, tickangle=45, ticks = 'outside', showgrid = False)
fig.show()

In [131]:
import pydeck as pdk

CPU_GRID_LAYER_DATA = (
    "https://raw.githubusercontent.com/uber-common/" "deck.gl-data/master/website/sf-bike-parking.json"
)
df = pd.read_json(CPU_GRID_LAYER_DATA)

# Define a layer to display on a map

layer = pdk.Layer(
    "GridLayer", df, pickable=True, extruded=True, cell_size=250, elevation_scale=5, get_position="COORDINATES",
)

view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=11, bearing=0, pitch=45)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "{position}\nCount: {count}"},)
r.to_html("grid_layer.html")

In [121]:
df["COORDINATES"]

0       [-122.42177834, 37.78346622]
1           [-122.414411, 37.774458]
2           [-122.438887, 37.772737]
3       [-122.42019976, 37.75087429]
4           [-122.392606, 37.779369]
                    ...             
2515    [-122.42152025, 37.75825474]
2516        [-122.432136, 37.767817]
2517    [-122.40502124, 37.79653486]
2518        [-122.423112, 37.782316]
2519        [-122.408778, 37.782887]
Name: COORDINATES, Length: 2520, dtype: object

In [185]:
count = pd.read_csv('totalridesfromstart.csv')
coord = pd.read_csv('stationlatlng.csv')
countlatlng = pd.merge(count, coord, on = 'start_station_id', how = 'inner')
countlatlng = countlatlng[countlatlng['start_lat'] != 0]
countlatlng['coordinates'] = countlatlng[['start_lon','start_lat']].values.tolist()
final = countlatlng[['count', 'coordinates']]


In [188]:


layer = pdk.Layer(
    "GridLayer", final, pickable=False, extruded=True, cell_size=500, elevation_scale = 'count', get_position = "coordinates"
)

view_state = pdk.ViewState(latitude=40.7, longitude=-73.95, zoom=10, bearing=0, pitch=45)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "{position}\nCount: {count}"},)
r

{"initialViewState": {"bearing": 0, "latitude": 40.7, "longitude": -73.95, "pitch": 45, "zoom": 10}, "layers": [{"@@type": "GridLayer", "cellSize": 500, "data": [{"coordinates": [-73.977706, 40.751873], "count": 950058}, {"coordinates": [-73.99009296, 40.73704984], "count": 759542}, {"coordinates": [-73.99415556, 40.74173969], "count": 732355}, {"coordinates": [-74.01322069, 40.71754834], "count": 694146}, {"coordinates": [-73.98955109, 40.7403432], "count": 674669}, {"coordinates": [-73.99074142, 40.73454567], "count": 638449}, {"coordinates": [-73.99102628231049, 40.730206605299536], "count": 600480}, {"coordinates": [-73.993934, 40.751551], "count": 592472}, {"coordinates": [-73.98191841, 40.76915505], "count": 574045}, {"coordinates": [-74.00711384, 40.73291553], "count": 559379}, {"coordinates": [-73.9900262, 40.75640548], "count": 559358}, {"coordinates": [-74.00277668, 40.76087502], "count": 558534}, {"coordinates": [-73.99724900722502, 40.722103786686034], "count": 554288}, {"coordinates": [-74.00214988, 40.73038599], "count": 546525}, {"coordinates": [-74.007756, 40.746745], "count": 540006}, {"coordinates": [-73.97634151, 40.76590936], "count": 500894}, {"coordinates": [-73.9916, 40.749156], "count": 487726}, {"coordinates": [-73.97371465, 40.7643971], "count": 471570}, {"coordinates": [-73.99093085, 40.75019995], "count": 467281}, {"coordinates": [-73.98848395, 40.74901271], "count": 464649}, {"coordinates": [-74.01658354, 40.7153379], "count": 464563}, {"coordinates": [-73.99138152, 40.75466591], "count": 463384}, {"coordinates": [-73.99456405, 40.73971301], "count": 463341}, {"coordinates": [-73.98165557, 40.73221853], "count": 461645}, {"coordinates": [-74.0026376103, 40.7390169121], "count": 457007}, {"coordinates": [-73.99529885, 40.74487634], "count": 452268}, {"coordinates": [-74.0046073, 40.71273266], "count": 441527}, {"coordinates": [-73.99379025, 40.72743423], "count": 438030}, {"coordinates": [-74.015756, 40.711512], "count": 435679}, {"coordinates": [-74.00451887, 40.74691959], "count": 432419}, {"coordinates": [-73.97498696, 40.75510267], "count": 429431}, {"coordinates": [-73.98142006, 40.72740794], "count": 423414}, {"coordinates": [-73.99200509, 40.73492695], "count": 423216}, {"coordinates": [-74.00674436, 40.73172428], "count": 422282}, {"coordinates": [-73.98379855, 40.72621788], "count": 421296}, {"coordinates": [-74.00297088, 40.72710258], "count": 420473}, {"coordinates": [-73.98194829, 40.7457121], "count": 419593}, {"coordinates": [-74.00197139, 40.7454973], "count": 418412}, {"coordinates": [-73.9946848154068, 40.7505853470215], "count": 407987}, {"coordinates": [-74.00293877, 40.73401143], "count": 407295}, {"coordinates": [-73.993915, 40.746647], "count": 402565}, {"coordinates": [-73.98658032, 40.75513557], "count": 401910}, {"coordinates": [-73.99723551, 40.74734825], "count": 396726}, {"coordinates": [-73.99931783, 40.73935542], "count": 393920}, {"coordinates": [-73.99238967, 40.73726186], "count": 391808}, {"coordinates": [-73.9912551, 40.76019252], "count": 388780}, {"coordinates": [-73.99915362, 40.74475148], "count": 387213}, {"coordinates": [-73.9845272898674, 40.76068327096592], "count": 385082}, {"coordinates": [-73.98855723, 40.7462009], "count": 382752}, {"coordinates": [-73.9734419, 40.74780373], "count": 382175}, {"coordinates": [-73.98192338, 40.7652654], "count": 379850}, {"coordinates": [-73.99144871, 40.74395411], "count": 379692}, {"coordinates": [-73.98672378, 40.73047309], "count": 379640}, {"coordinates": [-73.97973776, 40.73912601], "count": 374812}, {"coordinates": [-73.97706058, 40.74290902], "count": 373704}, {"coordinates": [-73.986923, 40.734232], "count": 373546}, {"coordinates": [-74.005226, 40.751396], "count": 370143}, {"coordinates": [-74.008591, 40.728846], "count": 370117}, {"coordinates": [-73.97791, 40.751581], "count": 369710}, {"coordinates": [-73.98713956, 40.7284186], "count": 368872}, {"coordinates": [-73.99480012, 40.72317958], "count": 367467},

In [146]:
df

,ADDRESS,RACKS,SPACES,COORDINATES
0,939 ELLIS ST,2,4,"[-122.42177834, 37.78346622]"
1,1380 HOWARD ST,1,2,"[-122.414411, 37.774458]"
2,1195 OAK ST,1,2,"[-122.438887, 37.772737]"
3,1387 VALENCIA ST,1,2,"[-122.42019976, 37.75087429]"
4,180 TOWNSEND ST,1,2,"[-122.392606, 37.779369]"
...,...,...,...,...
2515,900 VALENCIA ST,1,2,"[-122.42152025, 37.75825474]"
2516,91 WALTER ST,1,2,"[-122.432136, 37.767817]"
2517,916 KEARNY,1,2,"[-122.40502124, 37.79653486]"
2518,939 EDDY ST,1,2,"[-122.423112, 37.782316]"


In [155]:
count['count'].sum()

112546624

In [169]:
import pydeck as pdk
import pandas as pd

CPU_GRID_LAYER_DATA = (
    "https://raw.githubusercontent.com/uber-common/" "deck.gl-data/master/website/sf-bike-parking.json"
)
df = pd.read_json(CPU_GRID_LAYER_DATA)

# Define a layer to display on a map

layer = pdk.Layer(
    "GridLayer", df, pickable=True, extruded=True, cell_size=200, elevation_scale=4, get_position="COORDINATES",
)

view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=11, bearing=0, pitch=45)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={"text": "{position}\nCount: {count}"},)
r

{"initialViewState": {"bearing": 0, "latitude": 37.7749295, "longitude": -122.4194155, "pitch": 45, "zoom": 11}, "layers": [{"@@type": "GridLayer", "cellSize": 200, "data": [{"ADDRESS": "939 ELLIS ST", "COORDINATES": [-122.42177834, 37.78346622], "RACKS": 2, "SPACES": 4}, {"ADDRESS": "1380 HOWARD ST", "COORDINATES": [-122.414411, 37.774458], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "1195 OAK ST", "COORDINATES": [-122.438887, 37.772737], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "1387 VALENCIA ST", "COORDINATES": [-122.42019976, 37.75087429], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "180 TOWNSEND ST", "COORDINATES": [-122.392606, 37.779369], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "247 FILLMORE ST", "COORDINATES": [-122.43065953, 37.77185018], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "247 FILLMORE ST", "COORDINATES": [-122.43065953, 37.77185018], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "2690 MISSION ST", "COORDINATES": [-122.418974, 37.754029], "RACKS": 2, "SPACES": 4}, {"ADDRESS": "400 MCALLISTER ST", "COORDINATES": [-122.419014, 37.780519], "RACKS": 7, "SPACES": 14}, {"ADDRESS": "680 08TH ST", "COORDINATES": [-122.404719, 37.770128], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "101 TOWNSEND ST", "COORDINATES": [-122.390466, 37.780226], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "1186 FOLSOM ST", "COORDINATES": [-122.409866, 37.77547], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "1301 SANSOME ST", "COORDINATES": [-122.403298, 37.802327], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "1304 VALENCIA ST", "COORDINATES": [-122.420935, 37.751851], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "1380 VALENCIA ST", "COORDINATES": [-122.420834, 37.750802], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "1601 HOWARD ST", "COORDINATES": [-122.416789, 37.771394], "RACKS": 2, "SPACES": 4}, {"ADDRESS": "1700 FILBERT ST", "COORDINATES": [-122.427674, 37.799281], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "1700 OCEAN AVE", "COORDINATES": [-122.460192, 37.724988], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "201 GUERRERO ST", "COORDINATES": [-122.424167, 37.767853], "RACKS": 2, "SPACES": 4}, {"ADDRESS": "2500 16TH ST", "COORDINATES": [-122.411968, 37.76582], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "2525 16TH ST", "COORDINATES": [-122.411897, 37.765121], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "300 GOUGH ST", "COORDINATES": [-122.422666, 37.776104], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "3232 16TH ST", "COORDINATES": [-122.424904, 37.76499], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "425 DIVISADERO ST", "COORDINATES": [-122.437875, 37.773364], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "426 BRANNAN ST", "COORDINATES": [-122.395232, 37.779654], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "499 GUERRERO ST", "COORDINATES": [-122.42373, 37.763298], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "501 DOLORES ST", "COORDINATES": [-122.425711, 37.76132], "RACKS": 2, "SPACES": 4}, {"ADDRESS": "647 VALENCIA ST", "COORDINATES": [-122.42145, 37.762535], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "762 FULTON ST", "COORDINATES": [-122.429259, 37.778282], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "1 DR CARLTON B GOODLETT PL", "COORDINATES": [-122.41903, 37.778846], "RACKS": 2, "SPACES": 4}, {"ADDRESS": "1000 MARKET ST", "COORDINATES": [-122.410989, 37.782044], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "1026 VALENCIA ST", "COORDINATES": [-122.421372, 37.756417], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "117 06TH ST", "COORDINATES": [-122.408056, 37.780857], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "117 06TH ST", "COORDINATES": [-122.408056, 37.780857], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "1200 IRVING ST", "COORDINATES": [-122.471168, 37.764349], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "1201 OCEAN AVE", "COORDINATES": [-122.455509, 37.723447], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "1231 MARKET ST", "COORDINATES": [-122.414791, 37.777947], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "1235 MISSION ST", "COORDINATES": [-122.4135214, 37.77666033], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "130 SUTTER ST", "COORDINATES": [-122.40304, 37.790237], "RACKS": 1, "SPACES": 2}, {"ADDRESS": "1318 

In [ ]:
df[df['C']]